# 半梯度TD（Semi-gradient TD）的更新过程

半梯度TD是用于函数近似情况下的强化学习方法，之所以叫半梯度，是因为在更新过程中只考虑了改变权重向量w_t对于估计的影响，而不考虑对目标值的影响（即梯度值）。

在使用函数近似的情况下，我们用参数化函数 V(s,θ) 来近似真实的值函数 v_π(s)，其中 θ 是可调参数向量。

### 半梯度TD更新公式

$$ \theta_{t+1} = \theta_t + \alpha \cdot \delta_t \cdot \nabla_\theta V(s_t,\theta_t) $$

其中